In [1]:
%pip install nest_asyncio
import re

# Allow backtests to run in notebook
import nest_asyncio

nest_asyncio.apply()

# Increase width of console output for browsers
from IPython.display import HTML, display

display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

You should consider upgrading via the '/home/raphael/PycharmProjects/lazyft_lib/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

from lazyft import backtest, paths, remote
from lazyft.backtest.runner import BacktestMultiRunner
from lazyft.command_parameters import BacktestParameters, HyperoptParameters
from lazyft.config import Config
from lazyft.models import HyperoptReport
from lazyft.notify import notify_telegram
from lazyft.pairlist import refresh_pairlist
from lazyft.plot import plot_equity_curves
from lazyft.reports import get_backtest_repo, get_hyperopt_repo

2022-05-30 19:41:06.468 | WARNING  | lazyft.remote:<module>:25 - /home/raphael/PycharmProjects/lazyft_lib/remotes.json does not exist. Remotes not loaded


# Run a Backtest

In [3]:
b_timerange = "20220101-20220131"
# b_days = 30

b_conf = Config('config.json')

b_params = BacktestParameters(
    timerange=b_timerange,
    interval='1h',
    config_path=b_conf,
    # days=b_days,
    # secrets_config=priv_bin_us,
    # pairs=['PYR/USDT'],
    stake_amount='33.33',
    # timeframe_detail='5m',
    starting_balance=100,
    max_open_trades=3,
    download_data=True,
    tag='',
)
print(len(b_params.pairs), 'pairs')
# b_params.max_open_trades = len(b_params.pairs)
# print('strategy:', b_params.strategies[0])
print('max open trades:', b_params.max_open_trades)
print('timeframe:', b_params.timerange)
print('starting balance:', b_params.starting_balance)
print('exchange:', b_params.config['exchange']['name'])
print('timeframe:', b_params.interval)
if b_params.ensemble:
    print('ensemble:', b_params.ensemble)

30 pairs
max open trades: 3
timeframe: 20220101-20220131
starting balance: 100
exchange: binance
timeframe: 1h


In [4]:
runner = b_params.run('Hodl', load_from_hash=False)

2022-05-30 19:41:14.712 | INFO     | lazyft.parameter_tools:remove_params_file:36 - Removing strategy params: /home/raphael/PycharmProjects/lazyft_lib/user_data/strategies/hodl.json
2022-05-30 19:41:14.732 | INFO     | lazyft.strategy:save_strategy_text_to_database:203 - Saved strategy Hodl with hash 64b3362838ee2388cd2fc01d6c918994 to database
2022-05-30 19:41:16.438 | INFO     | lazyft.downloader:download_missing_historical_data:315 - Checking if download is needed for ROSE/USDT, MANA/USDT, DOT/USDT, YFII/USDT, SHIB/USDT, XRP/USDT, LRC/USDT, ETC/USDT, BUSD/USDT, DOGE/USDT, ADA/USDT, TRX/USDT, BTC/USDT, NEAR/USDT, BURGER/USDT, ETH/USDT, RAD/USDT, USDC/USDT, AVAX/USDT, GMT/USDT, GALA/USDT, CHESS/USDT, APE/USDT, EUR/USDT, ANC/USDT, MATIC/USDT, FTM/USDT, XMR/USDT, SAND/USDT, SOL/USDT, BTC/USDT, ETH/USDT, BUSD/USDT, USDC/USDT, XRP/USDT, SOL/USDT, ANC/USDT, GMT/USDT, TRX/USDT, SAND/USDT, AVAX/USDT, FTM/USDT, APE/USDT, NEAR/USDT, DOT/USDT, ADA/USDT, YFII/USDT, MANA/USDT, GALA/USDT, SHIB/USD

In [4]:
runner = b_params.run('Hodl', load_from_hash=False)

2022-05-30 19:32:23.232 | INFO     | lazyft.parameter_tools:remove_params_file:36 - Removing strategy params: /home/raphael/PycharmProjects/lazyft_lib/user_data/strategies/hodl.json
2022-05-30 19:32:23.246 | INFO     | lazyft.strategy:save_strategy_text_to_database:200 - Strategy Hodl already in database...skipping
2022-05-30 19:32:24.066 | INFO     | lazyft.downloader:download_missing_historical_data:315 - Checking if download is needed for GALA/USDT, ETC/USDT, TRX/USDT, MANA/USDT, BURGER/USDT, ROSE/USDT, ANC/USDT, SOL/USDT, APE/USDT, LRC/USDT, BTC/USDT, AVAX/USDT, EUR/USDT, CHESS/USDT, YFII/USDT, GMT/USDT, MATIC/USDT, BUSD/USDT, SAND/USDT, XRP/USDT, NEAR/USDT, USDC/USDT, DOT/USDT, SHIB/USDT, DOGE/USDT, ETH/USDT, ADA/USDT, RAD/USDT, FTM/USDT, XMR/USDT, BTC/USDT, ETH/USDT, BUSD/USDT, USDC/USDT, XRP/USDT, SOL/USDT, ANC/USDT, GMT/USDT, TRX/USDT, SAND/USDT, AVAX/USDT, FTM/USDT, APE/USDT, NEAR/USDT, DOT/USDT, ADA/USDT, YFII/USDT, MANA/USDT, GALA/USDT, SHIB/USDT, LRC/USDT, RAD/USDT, MATIC/U

In [5]:
runner.report.df

,id,strategy,hyperopt_id,date,exchange,m_o_t,stake,balance,n_pairlist,avg_profit_pct,avg_duration,wins,losses,sortino,drawdown,total_profit_pct,total_profit,trades,days,tag
0,None,Hodl,,05/30/22 19:41:18,binance,3,33.330,100,28,-0.246,"29 days, 23:00:00",0,2,-19.105,0.108,-0.164,-16.420,2,30,20220101-20220131
